In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from scipy.stats import trim_mean
from numpy import mean, absolute
import random

data = pd.read_csv('../data/weatherAUS.csv')

In [4]:
# Crear una copia del DataFrame con las columnas deseadas
df_copia = data.copy()


df_sin_nulos = df_copia.dropna(subset=['Sunshine'])
df_sin_nulos = df_sin_nulos.dropna(subset=['Evaporation'])


print('Number of instances = %d' % (df_sin_nulos.shape[0]))
print('Number of attributes = %d' % (df_sin_nulos.shape[1]))
print('---------------------------------------------------\n')
print('Number of missing values:')
for col in df_sin_nulos.columns:
    num_missing = df_sin_nulos[col].isna().sum()
    percentage_missing = (num_missing / df_sin_nulos.shape[0]) * 100
    print('\t%s: %d missing (%.3f%%) %s' % (col, num_missing, percentage_missing, df_sin_nulos[col].dtype))


Number of instances = 71382
Number of attributes = 23
---------------------------------------------------

Number of missing values:
	Date: 0 missing (0.000%) object
	Location: 0 missing (0.000%) object
	MinTemp: 617 missing (0.864%) float64
	MaxTemp: 589 missing (0.825%) float64
	Rainfall: 911 missing (1.276%) float64
	Evaporation: 0 missing (0.000%) float64
	Sunshine: 0 missing (0.000%) float64
	WindGustDir: 4351 missing (6.095%) object
	WindGustSpeed: 4328 missing (6.063%) float64
	WindDir9am: 2617 missing (3.666%) object
	WindDir3pm: 718 missing (1.006%) object
	WindSpeed9am: 215 missing (0.301%) float64
	WindSpeed3pm: 432 missing (0.605%) float64
	Humidity9am: 913 missing (1.279%) float64
	Humidity3pm: 1089 missing (1.526%) float64
	Pressure9am: 653 missing (0.915%) float64
	Pressure3pm: 660 missing (0.925%) float64
	Cloud9am: 7052 missing (9.879%) float64
	Cloud3pm: 8581 missing (12.021%) float64
	Temp9am: 652 missing (0.913%) float64
	Temp3pm: 847 missing (1.187%) float64
	RainT

In [ ]:
# Filtrar solo las columnas numéricas
columnas_numericas = data.select_dtypes(include=['number']).columns

# Agregar la columna categórica 'RainTomorrow'
columnas_a_incluir = list(columnas_numericas) + ['RainTomorrow']

# Crear una copia del DataFrame con las columnas deseadas
df_copia = data[columnas_a_incluir].copy()

# df_sin_nulos = df_copia.dropna(subset=['Sunshine'])
# df_sin_nulos = df_sin_nulos.dropna(subset=['Evaporation'])

df_sin_nulos =df_copia

withNullColumn = []

df_sin_nulos = df_sin_nulos.replace('NA', np.NaN)


for col in df_sin_nulos.columns:
    num_missing = df_sin_nulos[col].isna().sum()
    percentage_missing = (num_missing / df_sin_nulos.shape[0]) * 100
    print('\t%s: %d missing (%.3f%%) %s' % (col, num_missing, percentage_missing, df_sin_nulos[col].dtype))
    if num_missing > 0:
        withNullColumn.append((col))

# Iterar sobre las columnas con valores nulos
for col in withNullColumn:
    if df_sin_nulos[col].dtype == 'float64':
        # Reemplazar valores nulos en columnas float64 con la mediana de la columna
        median = df_sin_nulos[col].mean()
        df_sin_nulos[col].fillna(median, inplace=True)
    # # elif df_sin_nulos[col].dtype == 'object':
    # else:
    #     # Reemplazar valores nulos en columnas object con la moda de la columna
    #     mode = df_sin_nulos[col].mode()
    #     print('mode on col', col, mode[0])
    #     df_sin_nulos[col].fillna(mode[0], inplace=True)

withNullColumn = []

data2 = data.replace('NA', np.NaN)

print('Number of instances = %d' % (data2.shape[0]))
print('Number of attributes = %d' % (data2.shape[1]))
print('---------------------------------------------------\n')
print('Number of missing values:')
for col in data2.columns:
    num_missing = data2[col].isna().sum()
    percentage_missing = (num_missing / data2.shape[0]) * 100
    print('\t%s: %d missing (%.3f%%) %s' % (col, num_missing, percentage_missing, data2[col].dtype))
    if num_missing > 0:
        withNullColumn.append((col))


print(data.shape)
print(df_sin_nulos.shape)


#ranking de variables --------------------------------------------------------------------------------------------------

#Sacar las medias y las varianzas
means = df_sin_nulos.groupby("RainTomorrow").mean()
variances = df_sin_nulos.groupby("RainTomorrow").var()

# Calcular la diferencia absoluta entre los medias de RainTomorrow
dif_means = (means.loc["No"] - means.loc["Yes"]).abs()

# Calcular la diferencia absoluta entre las varianzas de RainTomorrow
dif_var = (variances.loc["No"] - variances.loc["Yes"]).abs()

rank_means = dif_means.rank(ascending=False) #Se de deja ascending=False ya que queremos que las diferencias más altas ocupen los primeros lugars
rank_var = dif_var.rank()

#unir los dos rankings y calcular el ranking final para cada dimensión
final_rank = ((rank_means + rank_var) /2).rank().sort_values()

print("Ranking de importancia de las dimensiones:")
print(final_rank)